This notebook contains code for traning a RandomForest regressor to predict measurement biases of miRXplore datasets from TGIRT-seq.

The [R RandomForest](https://cran.r-project.org/web/packages/randomForest/randomForest.pdf) model is being used here becuase neither [scikit-learn randomforest](https://github.com/scikit-learn/scikit-learn/issues/5442) nor the *train* module from [caret](https://stats.stackexchange.com/questions/135671/how-does-caret-handle-factors) can handle categorically-labeled data.

In [19]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
import seaborn as sns
from sequencing_tools.viz_tools import color_encoder, simpsons_palette, mixed_sort
from helper_function import *
from feature_selection import make_pca_df, pca_biplot, loading_plot, plot_outliers,\
                        labeling_expression
from rf_modeling import h2o_randomForest, R_randomForest, test_nucleotides, train_to_cat,\
                        rename_col, k_fold_cv, plot_kfold_reg, plot_var,\
                        plot_R2, plot_test

plt.rc('axes', labelsize=15)
plt.rc('xtick', labelsize=15)
plt.rc('ytick', labelsize=15)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Reading in miRNA count data, combining counts from replicates and only look at NTT data, the first and last 3 nucleotides of each miRNA are extracted as predictors, $\Delta log10$ CPM is computed and will be used as target:

In [2]:
df = pd.read_csv('../data/miR_counts.csv') \
    .rename(columns = {'id':'seq_id'})\
    .pipe(pd.melt, var_name = 'prep', value_name = 'seq_count', id_vars='seq_id')\
    .assign(prep = lambda d: d.prep.str.replace('[0-9]+$',''))\
    .groupby(["prep","seq_id"], as_index=False) \
    .agg({'seq_count':'sum'})\
    .merge(get_seq_base(shuffle = [0,1,2,-3,-2,-1]))\
    .assign(cpm = lambda d: d.groupby('prep').seq_count.transform(count_to_cpm))\
    .assign(expected_cpm = lambda d: 1e6 / 962) \
    .assign(Y = lambda d: np.log10(d['cpm']) - np.log10(d['expected_cpm']))  \
    .query('prep == "NTT"')\
    .reset_index() \
    .drop('index', axis=1)
df.head()

/Users/wckdouglas/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log10
  # Remove the CWD from sys.path while we load stuff.


,prep,seq_id,seq_count,head0,head1,head2,tail0,tail1,tail2,cpm,expected_cpm,Y
0,NTT,EBV-1-1,404.0,T,A,A,G,T,T,11.834411,1039.50104,-1.943678
1,NTT,EBV-1-2,42864.0,T,A,T,T,G,A,1255.619332,1039.50104,0.082033
2,NTT,EBV-1-2-star,2511.0,A,A,A,A,G,C,73.554968,1039.50104,-1.150213
3,NTT,EBV-1-3,3967.0,T,A,A,A,C,A,116.205718,1039.50104,-0.951597
4,NTT,EBV-1-3P,3687.0,T,A,G,G,T,C,108.003651,1039.50104,-0.983386


In [3]:
control_df = pd.read_csv('../data/miR_counts.csv') \
    .rename(columns = {'id':'seq_id'})\
    .pipe(pd.melt, var_name = 'prep', value_name = 'seq_count', id_vars='seq_id')\
    .assign(prep = lambda d: d.prep.str.replace('[0-9]+$',''))\
    .groupby(["prep","seq_id"], as_index=False) \
    .agg({'seq_count':'sum'})\
    .merge(get_seq_base(shuffle = [3,4,5,-6,-5,-4]))\
    .assign(cpm = lambda d: d.groupby('prep').seq_count.transform(count_to_cpm))\
    .assign(expected_cpm = lambda d: 1e6 / 962) \
    .assign(Y = lambda d: np.log10(d['cpm']) - np.log10(d['expected_cpm']))  \
    .query('prep == "NTT"')\
    .reset_index() \
    .drop('index', axis=1)
control_df.head()

/Users/wckdouglas/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log10
  # Remove the CWD from sys.path while we load stuff.


,prep,seq_id,seq_count,head0,head1,head2,tail0,tail1,tail2,cpm,expected_cpm,Y
0,NTT,EBV-1-1,404.0,C,C,T,G,G,A,11.834411,1039.50104,-1.943678
1,NTT,EBV-1-2,42864.0,C,T,T,A,A,T,1255.619332,1039.50104,0.082033
2,NTT,EBV-1-2-star,2511.0,T,T,C,G,A,T,73.554968,1039.50104,-1.150213
3,NTT,EBV-1-3,3967.0,C,G,G,A,G,C,116.205718,1039.50104,-0.951597
4,NTT,EBV-1-3P,3687.0,C,A,C,T,A,T,108.003651,1039.50104,-0.983386


In [6]:
model_df = df.filter(regex = 'head|tail|Y') \
    .pipe(train_to_cat)
model_df.head()

,head0,head1,head2,tail0,tail1,tail2,Y
0,T,A,A,G,T,T,-1.943678
1,T,A,T,T,G,A,0.082033
2,A,A,A,A,G,C,-1.150213
3,T,A,A,A,C,A,-0.951597
4,T,A,G,G,T,C,-0.983386


In [7]:
control_model_df = control_df.filter(regex = 'head|tail|Y') \
    .pipe(train_to_cat)
control_model_df.head()

,head0,head1,head2,tail0,tail1,tail2,Y
0,C,C,T,G,G,A,-1.943678
1,C,T,T,A,A,T,0.082033
2,T,T,C,G,A,T,-1.150213
3,C,G,G,A,G,C,-0.951597
4,C,A,C,T,A,T,-0.983386


# Train cross-validation #

In [ ]:
rrf = R_randomForest()
rf = GridSearchCV(estimator=rrf, 
             param_grid={'ntrees':np.arange(10,100,10),
                        'mtry':np.arange(2,10,1)},
             n_jobs = 4,
             refit = True,
             cv = 8, 
             return_train_score = True)
rf.fit(model_df.drop('Y', axis=1), model_df.Y)

/Users/wckdouglas/miniconda3/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [21]:
rf.cv_results_

{'mean_fit_time': array([0.09250776, 0.08357716, 0.12587579, 0.19233465, 0.17079059,
        0.19605454, 0.40262294, 0.26386746, 0.39354682, 0.12208835,
        0.11884467, 0.16859198, 0.21286782, 0.24723228, 0.26829775,
        0.47214397, 0.36602362, 0.37307668, 0.07710409, 0.12920435,
        0.19004544, 0.25789237, 0.36490766, 0.36568443, 0.55289602,
        0.53614163, 0.54712566, 0.08646663, 0.14346544, 0.20706232,
        0.26902215, 0.36957192, 0.38327575, 0.452775  , 0.516174  ,
        0.56712699, 0.09611066, 0.17088811, 0.24521629, 0.31507238,
        0.36976369, 0.44663461, 0.52218342, 0.62934232, 0.67102305,
        0.09311875, 0.16620628, 0.24090322, 0.32016969, 0.4533697 ,
        0.49203006, 0.91327437, 1.27504547, 1.54197049, 0.27745088,
        0.36350902, 0.5425059 , 0.61188896, 0.81536961, 0.93466369,
        0.69463102, 0.85686707, 0.88645466, 0.11418335, 0.3170987 ,
        0.32364194, 0.59016935, 0.49978765, 0.53104027, 0.71900535,
        0.70138272, 0.61681334]